In [ ]:
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair

In [ ]:
BDB_ROOT_URL = 'http://localhost:9984'
BDB = BigchainDB(BDB_ROOT_URL)

In [ ]:
uct = generate_keypair()

In [ ]:
def create_course(data, metadata, admin):
    check_course_data(data)
    check_course_metadata(metadata)
    transaction_id = process_creation(data, metadata, admin)
    return transaction_id
    
def process_creation(data, metadata, user):
    transaction = prepare_create_transaction(data, metadata, user.public_key)
    signed_transaction = fulfill_transaction(transaction, user.private_key)
    receipt = send_transaction(signed_transaction)
    if signed_transaction == receipt:
        return receipt.get('id')
    
def check_course_data(data):
    return True

def check_course_metadata(metadata):
    return True

def prepare_create_transaction(data, metadata, key):
    prepared_creation_tx = BDB.transactions.prepare(
        operation='CREATE',
        signers=key,
        asset=data,
        metadata=metadata,
    )
    return prepared_creation_tx

def fulfill_transaction(transaction, key):
    return BDB.transactions.fulfill(transaction, private_keys=key)

def send_transaction(transaction):
    return BDB.transactions.send_commit(transaction)

In [ ]:
course = {
    'data': {
        'course': {
            'name': 'Econometrics',
            'description': 'This course is an introductory course in Econometrics',
            'id': 'Econ104'
        }
    }
}
metadata = {
    'passing': 60,
    'distinction': 90
}

In [ ]:
econometrics_id = create_course(course, metadata, uct)

In [ ]:
BDB.assets.get(search='course')

In [ ]:
course = {
    'data': {
        'course': {
            'name': 'Statistics',
            'description': 'This course is an introductory course in Statistics',
            'id': 'Econ105'
        }
    }
}

metadata = {
    'passing': 60,
    'distinction': 90
}

In [ ]:
statistics_id = create_course(course, metadata, uct)

In [ ]:
BDB.assets.get(search='course')

In [ ]:
BDB.transactions.get(asset_id=econometrics_id)

In [ ]:
def create_degree(data, metadata, admin):
    check_degree_data(data)
    check_degree_metadata(metadata)
    transaction_id = process_creation(data, metadata, admin)
    return transaction_id
    
def check_degree_data(data):
    return True

def check_degree_metadata(metadata):
    return True

In [ ]:
degree = {
    'data': {
        'degree': {
            'name': 'Financial Technology',
            'level': 'Master',
            'description': 'Masters degree in fintech',
            'id': 'MFinTech'
        }
    }
}

metadata = {
    'courses': []
}

In [ ]:
fintech_id = create_degree(degree, metadata, uct)

In [ ]:
BDB.assets.get(search='degree')

In [ ]:
BDB.transactions.get(asset_id=fintech_id)

In [ ]:
def update_degree_course_list(asset_id, admin, course_id):
    tx, tx_id = get_last_transaction(asset_id)
    transaction_input = build_input(tx, tx_id)
    metadata = update_course_list(tx, course_id)
    transaction = prepare_update_transaction(asset_id, transaction_input, admin, metadata)
    signed_transaction = fulfill_transaction(transaction, admin.private_key)
    receipt = send_transaction(signed_transaction)
    if signed_transaction == receipt:
        return receipt.get('id')

def get_last_transaction(asset_id):
    transaction = BDB.transactions.get(asset_id=asset_id)[-1]
    transaction_id = transaction.get('id')
    return (transaction, transaction_id)

def build_input(tx, tx_id):
    output = tx.get('outputs')[-1]
    tx_input = {
        'fulfillment': output.get('condition').get('details'),
        'fulfills': {
            'output_index': 0,
            'transaction_id': tx_id,
        },
        'owners_before': output.get('public_keys'),
    }
    return tx_input

def update_course_list(tx, course_id):
    metadata = tx.get('metadata')
    metadata.get('courses').append(course_id)
    return metadata

def prepare_update_transaction(asset_id, tx_input, admin, metadata):
    tx_transfer = BDB.transactions.prepare(
        operation='TRANSFER',
        inputs=tx_input,
        asset={'id': asset_id},
        recipients=admin.public_key,
        metadata = metadata
    )
    return tx_transfer

In [ ]:
update_degree_course_list(fintech_id,
                          uct, 
                          econometrics_id)

In [ ]:
BDB.transactions.get(asset_id=fintech_id)

In [ ]:
update_degree_course_list(fintech_id,
                          uct, 
                          statistics_id)

In [ ]:
BDB.transactions.get(asset_id=fintech_id)

In [ ]:
def get_all_courses():
    return BDB.assets.get(search='course')

def get_all_degrees():
    return BDB.assets.get(search='degree')

In [ ]:
get_all_courses()

In [ ]:
get_all_degrees()

In [ ]:
def create_mark(data, metadata, admin):
    check_mark_data(data)
    check_mark_metadata(metadata)
    transaction_id = process_creation(data, metadata, admin)
    return transaction_id
    
def check_mark_data(data):
    return True

def check_mark_metadata(metadata):
    return True

In [ ]:
mark = {
    'data': {
        'mark': {
            'student': '0x03',
            'course': econometrics_id,
            'degree': fintech_id,
        }
    }
}

metadata = {
    'mark': 85
}

In [ ]:
mark_id = create_mark(mark, metadata, uct)

In [ ]:
def get_marks_by_address(address):
    marks = BDB.assets.get(search=address)
    for mark in marks:
        course_id = mark.get('data').get('mark').get('course')
        course_transaction = BDB.transactions.get(asset_id=course_id)
        course = course_transaction[0].get('asset').get('data').get('course').get('name')
        mark_id =  mark.get('id')
        mark_transaction = BDB.transactions.get(asset_id=mark_id)
        mark = mark_transaction[-1].get('metadata').get('mark')
        print(course, mark)

In [ ]:
get_marks_by_address('0x03')